In [1]:
import pandas
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
#from keras.utils import to_categorical
import feather
import random
import time
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif

C:\Users\aufbpc\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\aufbpc\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\aufbpc\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\aufbpc\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [2]:
FILENAME = "preproccesed_outliers_1085575_col_11_threshold_09"
data = feather.read_dataframe("{}.feather".format(FILENAME))

## Run this if you want to use the start latitude and longitude

In [ ]:
# Create one hot encoding labels
n = len(data)
labels = data['Severity']
labels_oh = np.zeros((n, 4))
for i in range(n):
    labels_oh[i, labels[i]-1] = 1
    
# Get the attributes
atts_oh = data.drop('Severity', axis=1)

In [ ]:
# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(atts_oh, labels_oh, test_size=0.33, random_state=30034)

In [ ]:
EPOCHS = 20
BATCH_SIZE = 128

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(30, activation=tf.nn.swish,input_shape=(X_train.shape[1],))) # Swish indeed performs better than sigmoid
model.add(tf.keras.layers.Dense(4, activation=tf.nn.softmax))

# Compile and print out summary of model
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
             metrics=['accuracy'])

model.summary()

# Feature Selection

In [4]:
# Split into train and test set
labels = data['Severity']
atts_oh = data.drop('Severity', axis=1)
X_train, X_test, y_train, y_test = train_test_split(atts_oh, labels, test_size=0.33, random_state=30034)

In [6]:
mi = SelectKBest(mutual_info_classif,k=100)
    
X_train_mi = mi.fit_transform(X_train,y_train)
X_test_mi = mi.transform(X_test)

In [ ]:
#one hot encoding y_train labels
n = len(y_train)
y_train_oh = np.zeros((n, 4))
for i in range(n):
    y_train_oh[i,y_train[i]-1] = 1

In [ ]:
# Train the model

start = time.time()

model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

end = time.time()
print(end - start)

In [ ]:
# Testing the model 
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\ntest_acc: ', test_acc)

## Remove start lat and start long

In [ ]:
new_data = data.drop(['Start_Lat', 'Start_Lng'], axis=1)

# Create one hot encoding labels
n = len(new_data)
labels = new_data['Severity']
labels_oh = np.zeros((n, 4))
for i in range(n):
    labels_oh[i, labels[i]-1] = 1
    
# Get the attributes
atts_oh = new_data.drop('Severity', axis=1)

In [ ]:
# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(atts_oh, labels_oh, test_size=0.33, random_state=30034)

In [ ]:
start = time.time()

model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

end = time.time()
print(end - start)

In [ ]:
# Testing the model 
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\ntest_acc: ', test_acc)

## (Just analysing the count of each weather conditions)

In [ ]:
count = 0
for col in data.columns:
    if col.startswith('Weather_Condition'):
        print(col[len('Weather_Condition_'):], sum(data[col]))
        count = count+1
print(count)